### SQL Interview Questions

using MySQL and Python

Installing mysql on mac is easy.

 - https://dev.mysql.com/doc/refman/5.7/en/macos-installation-pkg.html
 - https://dev.mysql.com/downloads/mysql/

You download Intel-based dmg installer (mysql-8.0.28-macos11-x86_64.dmg)
and follow instructions.

Note:
  About new Mac with M1 chip and python.
  As of this writing in January 2022 I recommend to use Intel-based installers.
  Native (to ARM M1 chip) version of python has appeared in July of 2021.
  But it is still too new, many modules are not supported yet. 
  Even today if you look at the official anaconda download page,
  there is no support for ARM.

Note:
<br>You will be asked to create root password during the installation.
<br>Or you can do it from terminal: 
``` code
mysqladmin -u root password YOURNEWPASSWORD
```

Add the root password into config file in your home directory:

```
vi ~/.my.cnf

[mysql]
user=root
password=YOURNEWPASSWORD
```

save - and change permissions:
```
chmod 0600 .my.cnf
```

edit .bashrc - add path to mysql executables like this:

```
PATH=$PATH:/usr/local/mysql/bin
```

Restart terminal.
Now you can start mysql prompt with "mysql" command

Once on mysql prompt, you can start working with SQL:

```
show databases;
create database testdb;
use testdb;
create table t1 ( i1 int null, c1 varchar(80) null );
show tables;
insert into t1 values (1,'a');
insert into t1 values (2,'b');
select * from t1;
exit
```

To work with mysql from python - look at this python file:

https://github.com/lselector/setup_computer/blob/master/py_lib/myutil_mysql.py

```
pip install mysql-connector-python
```

In [1]:
import os, sys

# add py_lib directory
myhome = os.getenv('HOME')
my_py_lib_dir = myhome+"/Documents/GitHub/setup_computer/py_lib"
if my_py_lib_dir not in sys.path:
    sys.path = [my_py_lib_dir] + sys.path

# add current directory to search path
if "." not in sys.path:
    sys.path = ["."] + sys.path

for p in sys.path:
    print("    ", p)

     .
     /Users/levselector/Documents/GitHub/setup_computer/py_lib
     /Users/levselector/Documents/GitHub/python_tutorials
     /Users/levselector/Documents/GitHub/python_tutorials/py_lib
     /Users/levselector/docs/py_lib
     /Users/levselector/miniconda3/lib/python39.zip
     /Users/levselector/miniconda3/lib/python3.9
     /Users/levselector/miniconda3/lib/python3.9/lib-dynload
     
     /Users/levselector/miniconda3/lib/python3.9/site-packages


In [2]:
import mysql.connector as connection
import pandas as pd

mysql_pwd = os.getenv('MYSQL_PWD')
mysql_conn = None # connection
try:
    mysql_conn = connection.connect(
        host     = "localhost", 
        database = 'testdb',
        user     = "root", 
        passwd   = mysql_pwd,
        use_pure = True)
    sql = "select * from t1;"
    df = pd.read_sql(sql, mysql_conn)
    print(df)
    # mysql_conn.close() # close the connection
except Exception as e:
    mysql_conn.close()
    print(str(e))

   i1     c1
0   1      a
1   2      b
2   5     cc
3   5     cc
4  10  NuxeI
5   2  DKXaH
6   0  mlcAO
7   3  aXplK


/Users/levselector/miniconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [3]:
import myutil_mysql
from myutil_mysql import *
dir(myutil_mysql)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'connect_to_mysql',
 'dbquote',
 'do_query',
 'do_sql',
 'dt',
 'mysql',
 'np',
 'os',
 'pd',
 're',
 'remove_extra_indents',
 'sys',
 'time']

In [4]:
# functions in myutil_mysql
# remove_extra_indents(sql) 
#    - returns SQL string without extra indents on the left
# dbquote(form, val)
#    - convenience function to format your value 
#      "val" into a string with quotes
# connect_to_mysql(myuser=None, mypasswd) 
#    - connects to the database
#    - returns tuple (cnx, 0) 
# do_query(cnx, sql) - runs sql query, 
#    - returns tuple (df, error_code)
# do_sql(cnx, sql) - runs sql, returns error code

In [5]:
# create connection
cnx,err = connect_to_mysql(myuser="root", mypasswd=os.getenv("MYSQL_PWD")) 

In [6]:
df,err = do_query(cnx, "select * from t1;")
print(df)
if err != 0:
    print("ERROR:",err)

   i1     c1
0   1      a
1   2      b
2   5     cc
3   5     cc
4  10  NuxeI
5   2  DKXaH
6   0  mlcAO
7   3  aXplK


/Users/levselector/miniconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
# example using do_query() function
df,err = do_query(cnx, "show tables;")
print(df)
if err != 0:
    print("ERROR:",err)

  Tables_in_testdb
0               t1


/Users/levselector/miniconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
# function to generate random string
import string
from random import choices, randint

In [9]:
def rstr(mylen=5):
    return ''.join(choices(string.ascii_letters,k=mylen))

In [10]:
# example using do_sql() function
ri = randint(0,10)
rs = rstr() # random string
err = do_sql(cnx, f"insert into t1 values({ri},'{rs}');")
if err != 0:
    print("ERROR:",err)

In [11]:
# example using do_query() function
df,err = do_query(cnx, "select * from t1 where c1 != 'cc';")
print(df)
if err != 0:
    print("ERROR:",err)

   i1     c1
0   1      a
1   2      b
2  10  NuxeI
3   2  DKXaH
4   0  mlcAO
5   3  aXplK
6   3  epPdh


/Users/levselector/miniconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [12]:
# mysql specific example - get current database and user
df,err = do_query(cnx, "select database()")
print(df)
print("-"*40)
df,err = do_query(cnx, "select user()")
print(df)

  database()
0     testdb
----------------------------------------
           user()
0  root@localhost


/Users/levselector/miniconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/levselector/miniconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [13]:
# mysql specific example - get current database
df,err = do_query(cnx, "show session status like 'Current%' ")
print(df.to_string())
if err != 0:
    print("ERROR:",err)

              Variable_name            Value
0            Current_tls_ca           ca.pem
1        Current_tls_capath                 
2          Current_tls_cert  server-cert.pem
3        Current_tls_cipher                 
4  Current_tls_ciphersuites                 
5           Current_tls_crl                 
6       Current_tls_crlpath                 
7           Current_tls_key   server-key.pem
8       Current_tls_version  TLSv1.2,TLSv1.3


/Users/levselector/miniconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
